In [13]:
# This cell should be exactly the same between all models

import pickle as pk

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from common import *
from util import *

# Load everything
scaled_data = {}
with np.load(PREPROCESSED_DATASET_FILEPATH) as npz_loader:
    for key in npz_loader:
        scaled_data[key] = npz_loader[key]
scaler = pk.load(open(PREPROCESSING_SCALER_FILEPATH, "rb"))

# Input and output dims
input_shape = tuple(list(scaled_data['x_train'].shape)[1:])
output_shape = tuple(list(scaled_data['y_train'].shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

In [14]:
print("tensorflow: {}".format(tf.__version__))
print("numpy: {}".format(np.__version__))

tensorflow: 2.8.0
numpy: 1.21.5


In [15]:
# CONSTANTS:
MODEL_NAME = "Feedforward"

In [24]:
# Construct TF model here

# Simple Feedforward Neural Network
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512,activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(0.001)),
#         tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(units=512,activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(0.001)),
#         tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(units=output_dims,activation='linear'),
        tf.keras.layers.Reshape(output_shape)
    ]
)


In [25]:
# hyperparameters and callbacks

BATCH_SIZE = 64
MAX_EPOCHS = 100

def scheduler(epoch, lr):
    if epoch<50:
        return lr
    elif epoch%20==0:
        return lr/10
    else:
        return lr
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(1e-3))

model(scaled_data['x_train'][:5])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (5, 4968)                 0         
                                                                 
 dense_15 (Dense)            (5, 512)                  2544128   
                                                                 
 dense_16 (Dense)            (5, 512)                  262656    
                                                                 
 dense_17 (Dense)            (5, 4968)                 2548584   
                                                                 
 reshape_5 (Reshape)         (5, 12, 207, 2)           0         
                                                                 
Total params: 5,355,368
Trainable params: 5,355,368
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile and fit model here
history = model.fit(
    x=scaled_data['x_train'],
    y=scaled_data['y_train'],
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCHS,
    validation_data=(scaled_data['x_val'],
    scaled_data['y_val']),
    callbacks=[callback_lr,callback_es]
)

Epoch 1/100
  1/375 [..............................] - ETA: 1:46 - loss: 2.2197

2022-04-21 12:30:24.525698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


372/375 [============================>.] - ETA: 0s - loss: 0.4553

2022-04-21 12:30:27.036120: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 3s 7ms/step - loss: 0.4543 - val_loss: 0.2798 - lr: 0.0010
Epoch 2/100
375/375 [==============================] - 2s 6ms/step - loss: 0.2466 - val_loss: 0.2315 - lr: 0.0010
Epoch 3/100
375/375 [==============================] - 2s 7ms/step - loss: 0.2292 - val_loss: 0.2263 - lr: 0.0010
Epoch 4/100
375/375 [==============================] - 2s 6ms/step - loss: 0.2219 - val_loss: 0.2262 - lr: 0.0010
Epoch 5/100
375/375 [==============================] - 2s 6ms/step - loss: 0.2157 - val_loss: 0.2242 - lr: 0.0010
Epoch 6/100
375/375 [==============================] - 2s 7ms/step - loss: 0.2127 - val_loss: 0.2269 - lr: 0.0010
Epoch 7/100
375/375 [==============================] - 2s 6ms/step - loss: 0.2094 - val_loss: 0.2149 - lr: 0.0010
Epoch 8/100
375/375 [==============================] - 2s 6ms/step - loss: 0.2044 - val_loss: 0.2130 - lr: 0.0010
Epoch 9/100
375/375 [==============================] - 2s 6ms/step - loss: 0.2025 - val_loss: 0.2095

In [21]:
# Compute unnormalized prediction loss

preds = {}

for split in ['test','val']:
    preds[split] = model.predict(scaled_data['x_'+split])
    preds[split] = scaler.inverse_transform(preds[split])

2022-04-21 12:30:03.659151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [22]:
# Save model
model.save(model_name_to_model_filepath(MODEL_NAME))

# Save run info
run_info = {}
run_info["history"] = history.history
run_info["predictions"] = preds # idk whether this part makes sense for RNNs or not
pk.dump(run_info, open(model_name_to_run_info_filepath(MODEL_NAME), "wb"))